In [2]:
from astropy.table import Table, vstack
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy import units as u
import subprocess as sp
import os

In [48]:
xy2skypath = '/Users/matthewkirby/software/wcstools/wcstools-3.9.5/bin/xy2sky'
megacat = Table()

# Set up the list of clusters for looping

In [21]:
def load_clusters():
    with open('personal_catalogs/cluster_info.dat') as fin:
        data = fin.readlines()[1:]

    # Load the cluster data
    data = [row.split() for row in data]
    cllist = [{'tablename': row[1], 'papername': row[2],
               'ra': float(row[3]), 'dec': float(row[4]), 'z': float(row[5]),
               'r200': float(row[6]), 'sigma': float(row[7]), 'sigma_hi': float(row[8]),
               'sigma_lo': float(row[9]), 'm200': float(row[10]), 'count': int(row[11]),
               'rinfall': float(row[12]), 'D_A': float(row[13])}
              for row in data]
    cltable = Table(cllist)
    
    return cltable

In [22]:
cltable = load_clusters()

In [58]:
clnames = sorted(list(set([cltable['tablename'][i].lower() for i in range(len(cltable))])))
catalog_names = ['ediscs_v7.0/{}_catalog_v7.0.fits'.format(name) for name in clnames]

image_path = '../ediscs_r_images/rims/'
image_names = np.array([image_path + '{}/{}_r_smooth.fits'.format(name, name) for name in clnames])
image_names[np.where(image_names == '../ediscs_r_images/rims/cl1420/cl1420_r_smooth.fits')] = image_path + 'cl1420/cl1420_r.fits'
image_names = image_names.tolist()

# Loop over each cluster and recalculate the positions

In [72]:
for catname, imname in zip(catalog_names, image_names):
    # Ensure that the two files are matched correctly
    print("Adding cluster {}".format(catname.split('/')[-1].split('_')[0]))
    if catname.split('/')[-1].split('_')[0] != imname.split('/')[-1].split('_')[0]:
        print("Catalog and image are not for the same clusters!")
    
    # Read in the catalog
    subcat = Table().read(catname)

    # Compute the RA and DEC using image wcs info
    np.savetxt('dummy.foobar', np.c_[(subcat['x'], subcat['y'])])
    sp.call('{} -d {} @dummy.foobar > dummy2.foobar 2>&1'.format(xy2skypath, imname), shell=True);
    subcat['ra'], subcat['dec'] = np.loadtxt('dummy2.foobar', usecols = (0, 1), unpack = True)

    # Remove the temporary files
    os.remove('dummy.foobar')
    os.remove('dummy2.foobar')
    
    # Throw it into the big catalog
    megacat = vstack([megacat, subcat])

Adding cluster cl1018
Adding cluster cl1037
Adding cluster cl1040
Adding cluster cl1054-11
Adding cluster cl1054-12
Adding cluster cl1059
Adding cluster cl1103
Adding cluster cl1138
Adding cluster cl1216
Adding cluster cl1227
Adding cluster cl1232
Adding cluster cl1301
Adding cluster cl1353
Adding cluster cl1354
Adding cluster cl1411
Adding cluster cl1420


In [74]:
megacat.write('personal_catalogs/megacat.fits', format='fits', overwrite=True)

# Look at what is in the original catalogs

In [5]:
catname = 'ediscs_v7.0/cl1018_catalog_v7.0.fits'
cat = Table().read(catname)['ra', 'dec']

In [14]:
print("{:10f}".format(cat[0]['ra']))
print("{:10f}".format(cat[0]['dec']))

154.613480
-12.463770


In [23]:
cat['ra'].astype(float)

154.6134796142578
154.57318115234375
154.6358642578125
154.7159423828125
154.54925537109375
154.87060546875
154.6569061279297
154.8824920654297
154.50497436523438
154.80445861816406
154.50022888183594


In [24]:
print("{:10f}".format(cat[0]['ra'].astype(float)))
print("{:10f}".format(cat[0]['dec']))

154.613480
-12.463770


In [25]:
cat[0]['ra'].astype(float)

154.6134796142578

In [31]:
d = 0.00001
d*3600

0.036000000000000004